In [2]:
# import libraries
import cv2
import mediapipe as mp
import numpy as np 

# Import MediaPipe Drawing module
mp_drawing = mp.solutions.drawing_utils
# Import MediaPipe Pose module
mp_pose = mp.solutions.pose

from angle_calculator import calculate_angle
from pose_utils import draw_keypoints_and_connections

In [3]:
# Pose Estimation in an image

# Define the path to image file 
image_path = 'curry.jpg'
# Read image from specified path


# Create pose object 
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5, model_complexity=1, static_image_mode=True) as pose:
    # Read image from specified path
    frame = cv2.imread(image_path)
    # Convert the image from BGR (OpenCV default) to RGB as required by MediaPipe.
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Prevent image from being written to improve performance
    image.flags.writeable = False
    
    # Process the image to detect pose landmarks
    results = pose.process(image)
   
    # Recolor back to BGR
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    # Render detections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                )               
        

    cv2.imshow('MediaPipe Detection', image)
    cv2.waitKey(0)  

cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:
# Extract Keypoints from an image

# Define the path to image file 
image_path = 'curry.jpg'
# Read image from specified path
frame = cv2.imread(image_path)
frame_height, frame_width, _ = frame.shape

# Create pose object 
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5, model_complexity=1, static_image_mode=True) as pose:
    # Convert the image from BGR (OpenCV default) to RGB as required by MediaPipe.
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Prevent image from being written to improve performance
    image.flags.writeable = False
    
    # Process the image to detect pose landmarks
    results = pose.process(image)
   
    # Recolor back to BGR
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Extract landmarks
    try:
        landmarks = results.pose_landmarks.landmark
    except:
        pass
    
    # Render detections
    draw_keypoints_and_connections(image, landmarks, 'all')  # Use 'right' or 'left' as needed     


    right_shoulder = np.rint(np.array([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x * frame_width,
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y * frame_height]))
    right_elbow = np.rint(np.array([landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x * frame_width,
                            landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y * frame_height]))
    right_wrist = np.rint(np.array([landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x * frame_width,
                            landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y * frame_height]))   

    

    a = right_shoulder - right_elbow
    b = right_wrist - right_elbow
    

    hip_angle = round(calculate_angle(a, b, 'elbow'))

  
    

    # Display the angle at the hip position in yellow with a smaller font size
    cv2.putText(image, str(hip_angle), 
                tuple(right_elbow.astype(int)), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1, cv2.LINE_AA)
    
    
    # Show the image
    cv2.imshow('Pose', image)
    
    cv2.waitKey(0)  

cv2.destroyAllWindows()


In [5]:
# Open video file stream for video capture
cap = cv2.VideoCapture('video_sample.avi')
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5, static_image_mode=False, model_complexity=2) as pose:
    # Loops through each frame in the video
    while cap.isOpened():
        # read frame
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
        landmarks = results.pose_landmarks.landmark
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Render detections
        draw_keypoints_and_connections(image, landmarks, 'right')  # Use 'right' or 'left' as needed           
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()